In [2]:
import re
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from collections import defaultdict

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
lemmatizer=WordNetLemmatizer()
stop_words=set(stopwords.words('english'))
def process_sentence(sentence):
    nouns=list()
    base_words=list()
    final_words=list()
    sentence=re.sub(r'[^ \w\s]','',sentence)
    sentence=re.sub(r'_',' ',sentence)
    words=word_tokenize(sentence)
    pos_tagged_words=pos_tag(words)
    for token,tag in pos_tagged_words:
        base_words.append(lemmatizer.lemmatize(token,tag_map[tag[0]]))
    for word in base_words:
        if word not in stop_words:
            final_words.append(word)
    sym=' '
    sent=sym.join(final_words)
    pos_tagged_sent=pos_tag(final_words)
    for token,tag in pos_tagged_sent:
        if tag=='NN':
            nouns.append(token)
    return sent,nouns
def process_email(email):
    email=email[9::]
    email=email.lower()
    sentences=sent_tokenize(email)
    total_nouns=list()
    em=""
    for sentence in sentences:
        sent,nouns=process_sentence(sentence)
        em+=" "+sent
        total_nouns+=nouns
    return em

In [3]:
import pandas as pd
df=pd.read_csv('Data.csv')
df.head()

,Body,Label
0,Subject: stock promo mover : cwtd\n * * * urge...,1
1,Subject: are you listed in major search engine...,1
2,"Subject: important information thu , 30 jun 20...",1
3,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,"Subject: "" bidstogo "" is places to go , things...",1


In [6]:
import numpy as np
df['Body']=df.Body.apply(lambda email:np.str_(email))

# Processing Emails

In [7]:
df['Email']=df.Body.apply(process_email)
df.head()

,Body,Label,Email
0,Subject: stock promo mover : cwtd\n * * * urge...,1,stock promo mover cwtd urgent investor tradin...
1,Subject: are you listed in major search engine...,1,list major search engine submit website searc...
2,"Subject: important information thu , 30 jun 20...",1,important information thu 30 jun 2005 subject...
3,Subject: = ? utf - 8 ? q ? bask your life with...,1,utf 8 q bask life utf 8 q individual increm...
4,"Subject: "" bidstogo "" is places to go , things...",1,bidstogo place go thing hello privacy policy ...


In [8]:
Data=df[['Email','Label']]
Data.head()

,Email,Label
0,stock promo mover cwtd urgent investor tradin...,1
1,list major search engine submit website searc...,1
2,important information thu 30 jun 2005 subject...,1
3,utf 8 q bask life utf 8 q individual increm...,1
4,bidstogo place go thing hello privacy policy ...,1


In [9]:
Data.Label.value_counts()

0    7095
1    5183
Name: Label, dtype: int64

# Converting Dataframe to CSV

In [10]:
Data.to_csv('Cleaned_Data.csv',index=False)